In [1]:
from pathlib import Path
from helper.read_config import read_config, get_prune_config
from helper.load_model import choose_device, load_predictor_from_folder
from prune.range_prune import apply_range_pruning_to_model
from prune.verify_prune import verify_pruning
from pprint import pprint
from prune.predict_after_prune import predict_after_prune

nnUNet_raw is not defined and nnU-Net can only be used on data for which preprocessed files are already present on your system. nnU-Net cannot be used for experiment planning and preprocessing like this. If this is not intended, please read documentation/setting_up_paths.md for information on how to set this up properly.
nnUNet_preprocessed is not defined and nnU-Net can not be used for preprocessing or training. If this is not intended, please read documentation/setting_up_paths.md for information on how to set this up.
nnUNet_results is not defined and nnU-Net cannot be used for training or inference. If this is not intended behavior, please read documentation/setting_up_paths.md for information on how to set this up.


In [2]:
config_path = Path("scripts/config/config.yaml")
device = choose_device()

In [3]:
# Get the network architecture
config = read_config(config_path)
prune_config = get_prune_config(config)
pprint(prune_config)
predictor = load_predictor_from_folder(prune_config['model_folder'], prune_config['fold_tuple'], prune_config['checkpoint_name'])
model = predictor.network

# Inspect state_dict BEFORE pruning
print("===== STATE DICT BEFORE PRUNING =====")
before_keys = set()
for key in model.state_dict().keys():
    before_keys.add(key)
    # Print a sample of keys to avoid overwhelming output
    if len(before_keys) <= 10 or 'encoder.stages.0.0.convs.0' in key:
        print(f"KEY: {key}")

{'checkpoint_name': 'checkpoint_final.pth',
 'fold': 0,
 'fold_tuple': (0,),
 'model_folder': 'nnUNet_results_prune_test/Dataset027_ACDC/FlexibleTrainerV1__nnUNetPlans__2d',
 'predict': {'input_folder': 'nnUNet_raw/Dataset027_ACDC/imagesTs',
             'output_folder': 'nnUNet_results_prune_test/Dataset027_ACDC/FlexibleTrainerV1__nnUNetPlans__2d/predictions_test2'},
 'prune_method': None}
/mnt/data/pruning_nnunet/nnunetv2/nnunetv2
Using label manager: <class 'nnunetv2.utilities.label_handling.label_handling.LabelManager'>
===== STATE DICT BEFORE PRUNING =====
KEY: encoder.stages.0.0.convs.0.conv.weight
KEY: encoder.stages.0.0.convs.0.conv.bias
KEY: encoder.stages.0.0.convs.0.norm.weight
KEY: encoder.stages.0.0.convs.0.norm.bias
KEY: encoder.stages.0.0.convs.0.all_modules.0.weight
KEY: encoder.stages.0.0.convs.0.all_modules.0.bias
KEY: encoder.stages.0.0.convs.0.all_modules.1.weight
KEY: encoder.stages.0.0.convs.0.all_modules.1.bias
KEY: encoder.stages.0.0.convs.1.conv.weight
KEY: enc

In [4]:
# print(list(model.named_parameters()))
# print(model)
verify_pruning(predictor.network)

No convolutional layers / weight masks found in the model.


False

In [5]:
prune_model = apply_range_pruning_to_model(model, min_val=-5.0e-0, max_val=5.0e-0)

# Inspect state_dict AFTER pruning
print("\n===== STATE DICT AFTER PRUNING =====")
after_keys = set()
for key in prune_model.state_dict().keys():
    after_keys.add(key)
    # Print a sample of keys to avoid overwhelming output
    if len(after_keys) <= 10 or 'encoder.stages.0.0.convs.0' in key:
        print(f"KEY: {key}")

encoder.stages.0.0.convs.0.conv: pruned 288/288 weights (100.00%)
encoder.stages.0.0.convs.1.conv: pruned 9216/9216 weights (100.00%)
encoder.stages.1.0.convs.0.conv: pruned 18432/18432 weights (100.00%)
encoder.stages.1.0.convs.1.conv: pruned 36864/36864 weights (100.00%)
encoder.stages.2.0.convs.0.conv: pruned 73728/73728 weights (100.00%)
encoder.stages.2.0.convs.1.conv: pruned 147456/147456 weights (100.00%)
encoder.stages.3.0.convs.0.conv: pruned 294912/294912 weights (100.00%)
encoder.stages.3.0.convs.1.conv: pruned 589824/589824 weights (100.00%)
encoder.stages.4.0.convs.0.conv: pruned 1179648/1179648 weights (100.00%)
encoder.stages.4.0.convs.1.conv: pruned 2359296/2359296 weights (100.00%)
encoder.stages.5.0.convs.0.conv: pruned 2359296/2359296 weights (100.00%)
encoder.stages.5.0.convs.1.conv: pruned 2359296/2359296 weights (100.00%)
decoder.stages.0.convs.0.conv: pruned 4718592/4718592 weights (100.00%)
decoder.stages.0.convs.1.conv: pruned 2359296/2359296 weights (100.00%)


In [6]:
# Show what's changed
print("\n===== CHANGES IN STATE DICT =====")
removed_keys = before_keys - after_keys
added_keys = after_keys - before_keys

print("REMOVED KEYS:")
for key in removed_keys:
    if 'encoder.stages.0.0.convs.0' in key:
        print(f"- {key}")

print("\nADDED KEYS:")
for key in added_keys:
    if 'encoder.stages.0.0.convs.0' in key:
        print(f"+ {key}")


===== CHANGES IN STATE DICT =====
REMOVED KEYS:
- encoder.stages.0.0.convs.0.conv.weight
- encoder.stages.0.0.convs.0.all_modules.0.weight
- decoder.encoder.stages.0.0.convs.0.conv.weight
- decoder.encoder.stages.0.0.convs.0.all_modules.0.weight

ADDED KEYS:
+ encoder.stages.0.0.convs.0.all_modules.0.weight_mask
+ decoder.encoder.stages.0.0.convs.0.conv.weight_orig
+ decoder.encoder.stages.0.0.convs.0.conv.weight_mask
+ encoder.stages.0.0.convs.0.conv.weight_orig
+ decoder.encoder.stages.0.0.convs.0.all_modules.0.weight_orig
+ encoder.stages.0.0.convs.0.all_modules.0.weight_orig
+ decoder.encoder.stages.0.0.convs.0.all_modules.0.weight_mask
+ encoder.stages.0.0.convs.0.conv.weight_mask


In [7]:
_ = verify_pruning(predictor.network)

Layer encoder.stages.0.0.convs.0.conv: 288/288 weights pruned (100.00% sparsity)
Layer encoder.stages.0.0.convs.1.conv: 9216/9216 weights pruned (100.00% sparsity)
Layer encoder.stages.1.0.convs.0.conv: 18432/18432 weights pruned (100.00% sparsity)
Layer encoder.stages.1.0.convs.1.conv: 36864/36864 weights pruned (100.00% sparsity)
Layer encoder.stages.2.0.convs.0.conv: 73728/73728 weights pruned (100.00% sparsity)
Layer encoder.stages.2.0.convs.1.conv: 147456/147456 weights pruned (100.00% sparsity)
Layer encoder.stages.3.0.convs.0.conv: 294912/294912 weights pruned (100.00% sparsity)
Layer encoder.stages.3.0.convs.1.conv: 589824/589824 weights pruned (100.00% sparsity)
Layer encoder.stages.4.0.convs.0.conv: 1179648/1179648 weights pruned (100.00% sparsity)
Layer encoder.stages.4.0.convs.1.conv: 2359296/2359296 weights pruned (100.00% sparsity)
Layer encoder.stages.5.0.convs.0.conv: 2359296/2359296 weights pruned (100.00% sparsity)
Layer encoder.stages.5.0.convs.1.conv: 2359296/235929

In [8]:
predict_after_prune(predictor, prune_config)

Saved pruned model with masks to: nnUNet_results_prune_test/Dataset027_ACDC/FlexibleTrainerV1__nnUNetPlans__2d/predictions_test2/fold_0/final_model/pruned_model_with_masks.pth
Removing pruning parameterization for Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Removing pruning parameterization for Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Removing pruning parameterization for Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
Removing pruning parameterization for Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Removing pruning parameterization for Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
Removing pruning parameterization for Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Removing pruning parameterization for Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
Removing pruning parameterization for Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), pa

NiftiImageIO (0x3cbb2ec0): nnUNet_raw/Dataset027_ACDC/imagesTs/patient101_frame01_0000.nii.gz has unexpected scales in sform

NiftiImageIO (0x3cbb2ec0): nnUNet_raw/Dataset027_ACDC/imagesTs/patient101_frame01_0000.nii.gz has unexpected scales in sform

NiftiImageIO (0x39b205d0): nnUNet_raw/Dataset027_ACDC/imagesTs/patient101_frame14_0000.nii.gz has unexpected scales in sform

NiftiImageIO (0x39b205d0): nnUNet_raw/Dataset027_ACDC/imagesTs/patient101_frame14_0000.nii.gz has unexpected scales in sform

NiftiImageIO (0x29d12df0): nnUNet_raw/Dataset027_ACDC/imagesTs/patient102_frame01_0000.nii.gz has unexpected scales in sform

NiftiImageIO (0x29d12df0): nnUNet_raw/Dataset027_ACDC/imagesTs/patient102_frame01_0000.nii.gz has unexpected scales in sform

NiftiImageIO (0x3ceeeae0): nnUNet_raw/Dataset027_ACDC/imagesTs/patient102_frame13_0000.nii.gz has unexpected scales in sform

NiftiImageIO (0x3ceeeae0): nnUNet_raw/Dataset027_ACDC/imagesTs/patient102_frame13_0000.nii.gz has unexpected scales in


Predicting patient101_frame01:
perform_everything_on_device: True
sending off prediction to background worker for resampling and export
done with patient101_frame01

Predicting patient101_frame14:
perform_everything_on_device: True


NiftiImageIO (0x39b205d0): nnUNet_raw/Dataset027_ACDC/imagesTs/patient104_frame11_0000.nii.gz has unexpected scales in sform

NiftiImageIO (0x39b205d0): nnUNet_raw/Dataset027_ACDC/imagesTs/patient104_frame11_0000.nii.gz has unexpected scales in sform



sending off prediction to background worker for resampling and export
done with patient101_frame14

Predicting patient102_frame01:
perform_everything_on_device: True
sending off prediction to background worker for resampling and export
done with patient102_frame01

Predicting patient102_frame13:
perform_everything_on_device: True


...

sending off prediction to background worker for resampling and export
done with patient145_frame13

Predicting patient146_frame01:
perform_everything_on_device: True
sending off prediction to background worker for resampling and export
done with patient146_frame01

Predicting patient146_frame10:
perform_everything_on_device: True
sending off prediction to background worker for resampling and export
done with patient146_frame10


NiftiImageIO (0x3cbb2ec0): nnUNet_raw/Dataset027_ACDC/imagesTs/patient149_frame01_0000.nii.gz has unexpected scales in sform

NiftiImageIO (0x3cbb2ec0): nnUNet_raw/Dataset027_ACDC/imagesTs/patient149_frame01_0000.nii.gz has unexpected scales in sform

NiftiImageIO (0x39b205d0): nnUNet_raw/Dataset027_ACDC/imagesTs/patient149_frame12_0000.nii.gz has unexpected scales in sform

NiftiImageIO (0x39b205d0): nnUNet_raw/Dataset027_ACDC/imagesTs/patient149_frame12_0000.nii.gz has unexpected scales in sform

NiftiImageIO (0x29d12df0): nnUNet_raw/Dataset027_ACDC/imagesTs/patient150_frame01_0000.nii.gz has unexpected scales in sform

NiftiImageIO (0x29d12df0): nnUNet_raw/Dataset027_ACDC/imagesTs/patient150_frame01_0000.nii.gz has unexpected scales in sform




Predicting patient147_frame01:
perform_everything_on_device: True
sending off prediction to background worker for resampling and export
done with patient147_frame01

Predicting patient147_frame09:
perform_everything_on_device: True


NiftiImageIO (0x3ceeeae0): nnUNet_raw/Dataset027_ACDC/imagesTs/patient150_frame12_0000.nii.gz has unexpected scales in sform

NiftiImageIO (0x3ceeeae0): nnUNet_raw/Dataset027_ACDC/imagesTs/patient150_frame12_0000.nii.gz has unexpected scales in sform



sending off prediction to background worker for resampling and export
done with patient147_frame09
nnUNet_raw is not defined and nnU-Net can only be used on data for which preprocessed files are already present on your system. nnU-Net cannot be used for experiment planning and preprocessing like this. If this is not intended, please read documentation/setting_up_paths.md for information on how to set this up properly.
nnUNet_preprocessed is not defined and nnU-Net can not be used for preprocessing or training. If this is not intended, please read documentation/setting_up_paths.md for information on how to set this up.
nnUNet_results is not defined and nnU-Net cannot be used for training or inference. If this is not intended behavior, please read documentation/setting_up_paths.md for information on how to set this up.
/mnt/data/pruning_nnunet/nnunetv2/nnunetv2
Using label manager: <class 'nnunetv2.utilities.label_handling.label_handling.LabelManager'>

Predicting patient148_frame01:
per